In [1]:
# Step 1: Install required libraries
!pip install pandas cassandra-driver

# Step 2: Load the CSV file
import pandas as pd

# Download the CSV file from GitHub
csv_url = "https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv"
data = pd.read_csv(csv_url)

# Display the first few rows
print("Sample data:")
print(data.head())

# Save to a local file for reference
data.to_csv('sales_100.csv', index=False)
print("Data saved locally as 'sales_100.csv'")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 27.4 MB/s eta 0:00:00
Sample data:
                         Region           Country  Item Type Sales Channel  \
0            Sub-Saharan Africa      South Africa     Fruits       Offline   
1  Middle East and North Africa           Morocco    Clothes        Online   
2         Australia and Oceania  Papua New Guinea       Meat       Offline   
3            Sub-Saharan Africa          Djibouti    Clothes       Offline   
4                        Europe          Slovakia  Beverages       Offline   

  Order Priority  Order Date   Order ID   Ship Date  UnitsSold  UnitPrice  \
0              M   7/27/2012  443368995   7/28/2012       1593       9.33   
1              M   9/14/2013  667593514  10/19/2013       4611     109.28   
2              M   5/15/2015  940995585    6/4/2015        360     421.89   
3              H   5/17/2017  880811536    7/2/2017        562     109.28   
4              L  10/26/2016  174590194   12/4/2016     

In [3]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

In [15]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd
import json

cloud_config= {
        'secure_connect_bundle': 'secure-connect-assignment1.zip'
}

# Load credentials
with open("Assignment1-token.json") as f:
    secrets = json.load(f)
CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if session:
  print("active")
else:
  print("An error occurred.")

active


In [38]:
create_table_query = """
CREATE TABLE learning.bronze_data (
    Region TEXT,
    Country TEXT,
    Item_Type TEXT,
    Sales_Channel TEXT,
    Order_Priority TEXT,
    Order_Date TEXT,
    Order_ID BIGINT PRIMARY KEY,
    Ship_Date TEXT,
    Units_Sold INT,
    Unit_Price FLOAT,
    Unit_Cost FLOAT,
    Total_Revenue FLOAT,
    Total_Cost FLOAT,
    Total_Profit FLOAT
);
"""
session.execute(create_table_query)
print("Table created successfully!")

AlreadyExists: Table 'learning.bronze_data' already exists

In [21]:
query = "SELECT * FROM learning.bronze_data ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(order_id=294530856, country='Italy', item_type='Cereal', order_date='11/15/2011', order_priority='M', region='Europe', sales_channel='Online', ship_date='12/28/2011', total_cost=829138.8125, total_profit=627217.1875, total_revenue=1456356.0, unit_cost=117.11000061035156, unit_price=205.6999969482422, units_sold=7080)
Row(order_id=274930989, country='Dominica', item_type='Household', order_date='11/19/2011', order_priority='C', region='Central America and the Caribbean', sales_channel='Offline', ship_date='12/13/2011', total_cost=3539891.75, total_profit=1167402.125, total_revenue=4707294.0, unit_cost=502.5400085449219, unit_price=668.27001953125, units_sold=7044)
Row(order_id=498071897, country='Taiwan', item_type='Cereal', order_date='4/11/2010', order_priority='H', region='Asia', sales_channel='Online', ship_date='5/26/2010', total_cost=1100482.625, total_profit=832480.25, total_revenue=1932962.875, unit_cost=117.11000061035156, unit_price=205.6999969482422, units_sold=9397)
Row(

In [23]:
create_silver_table_query = """
CREATE TABLE IF NOT EXISTS learning.silver_sales (
    region text,
    country text,
    item_type text,
    sales_channel text,
    order_priority text,
    order_date date,
    order_id UUID PRIMARY KEY,
    ship_date date,
    units_sold int,
    unit_price float,
    unit_cost float,
    total_revenue float,
    total_cost float,
    total_profit float
);

"""
session.execute(create_silver_table_query)
print("Table created successfully!")

Table created successfully!


In [43]:
import pandas as pd
from cassandra.cluster import Cluster
from datetime import datetime
import uuid

# Step 1: Load and clean the data
csv_file_path = "sales_100.csv"
df = pd.read_csv(csv_file_path)

df.columns = df.columns.str.strip()

print("Column names in the DataFrame:", df.columns)

df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%m/%d/%Y')

# Step 2: Define the required columns and check for missing ones
required_columns = ['Order ID', 'Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
                    'Order Date', 'Ship Date', 'UnitsSold', 'UnitPrice', 'UnitCost', 'TotalRevenue',
                    'TotalCost', 'TotalProfit']

missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"Missing columns in the DataFrame: {missing_columns}")
else:


    # Step 4: Insert each row into Cassandra using iterrows()
    for _, row in df.iterrows():
        query = """
        INSERT INTO learning.silver_sales (
            order_id, region, country, item_type, sales_channel, order_priority,
            order_date, ship_date, units_sold, unit_price, unit_cost, total_revenue,
            total_cost, total_profit
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Convert 'Order ID' to UUID if it's not already
        # Check if the 'Order ID' is a valid integer, and convert it to UUID
        if isinstance(row['Order ID'], int):
            order_id = uuid.uuid5(uuid.NAMESPACE_DNS, str(row['Order ID']))  # Generate UUID based on the integer
        else:
            order_id = uuid.UUID(row['Order ID'])  # If it's already a UUID string

        session.execute(query, (
            order_id,  # UUID type
            row['Region'],  # Region
            row['Country'],  # Country
            row['Item Type'],  # Item Type
            row['Sales Channel'],  # Sales Channel
            row['Order Priority'],  # Order Priority
            row['Order Date'].date(),  # 'Order Date' as DATE type
            row['Ship Date'].date(),  # 'Ship Date' as DATE type
            int(row['UnitsSold']),  # Units Sold
            float(row['UnitPrice']),  # Unit Price
            float(row['UnitCost']),  # Unit Cost
            float(row['TotalRevenue']),  # Total Revenue
            float(row['TotalCost']),  # Total Cost
            float(row['TotalProfit'])  # Total Profit
        ))

    print("Silver table populated successfully!")


Column names in the DataFrame: Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date', 'Order ID', 'Ship Date', 'UnitsSold', 'UnitPrice',
       'UnitCost', 'TotalRevenue', 'TotalCost', 'TotalProfit'],
      dtype='object')
Silver table populated successfully!


In [44]:
query = "SELECT * FROM learning.silver_sales ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(order_id=UUID('40d641b2-f5bc-5cfd-9261-0ec2561d71b0'), country='South Africa', item_type='Fruits', order_date=Date(15548), order_priority='M', region='Sub-Saharan Africa', sales_channel='Offline', ship_date=Date(15549), total_cost=11023.5595703125, total_profit=3839.1298828125, total_revenue=14862.6904296875, unit_cost=6.920000076293945, unit_price=9.329999923706055, units_sold=1593)
Row(order_id=UUID('472b20a1-e4ff-5b13-a9b7-b2a93efdca62'), country='United Arab Emirates', item_type='Office Supplies', order_date=Date(15514), order_priority='M', region='Middle East and North Africa', sales_channel='Online', ship_date=Date(15527), total_cost=5041191.0, total_profit=1212378.75, total_revenue=6253569.5, unit_cost=524.9600219726562, unit_price=651.2100219726562, units_sold=9603)
Row(order_id=UUID('0791d9b1-c215-5d51-be8d-7ed65918272d'), country='Vanuatu', item_type='Cereal', order_date=Date(16241), order_priority='C', region='Australia and Oceania', sales_channel='Offline', ship_date=Da

In [45]:
# Step 1: Create the gold table for total sales by region and item type
query_gold_create_1 = """
CREATE TABLE IF NOT EXISTS learning.gold_sales_by_region_item (
    region TEXT,
    item_type TEXT,
    total_revenue DECIMAL,
    total_units_sold INT,
    PRIMARY KEY (region, item_type)
)
"""
session.execute(query_gold_create_1)


In [49]:
from collections import defaultdict

# Fetch all relevant data from the silver_sales table
fetch_query = "SELECT region, item_type, total_revenue, units_sold FROM learning.silver_sales"
rows = session.execute(fetch_query)

# Perform aggregation in Python
aggregated_data = defaultdict(lambda: defaultdict(lambda: {'total_revenue': 0, 'total_units_sold': 0}))

for row in rows:
    aggregated_data[row.region][row.item_type]['total_revenue'] += row.total_revenue
    aggregated_data[row.region][row.item_type]['total_units_sold'] += row.units_sold

# Insert aggregated data into the gold table
for region, items in aggregated_data.items():
    for item_type, totals in items.items():
        session.execute("""
        INSERT INTO learning.gold_sales_by_region_item (region, item_type, total_revenue, total_units_sold)
        VALUES (%s, %s, %s, %s)
        """, (region, item_type, totals['total_revenue'], totals['total_units_sold']))

print("Gold table 'gold_sales_by_region_item' populated successfully!")

Gold table 'gold_sales_by_region_item' populated successfully!


In [50]:
query = "SELECT * FROM learning.gold_sales_by_region_item ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(region='Australia and Oceania', item_type='Baby Food', total_revenue=Decimal('1411953.625'), total_units_sold=5531)
Row(region='Australia and Oceania', item_type='Beverages', total_revenue=Decimal('274640.59375'), total_units_sold=5788)
Row(region='Australia and Oceania', item_type='Cereal', total_revenue=Decimal('1221858.05078125'), total_units_sold=5940)
Row(region='Australia and Oceania', item_type='Clothes', total_revenue=Decimal('993573.75'), total_units_sold=9092)
Row(region='Australia and Oceania', item_type='Fruits', total_revenue=Decimal('53507.55078125'), total_units_sold=5735)
Row(region='Australia and Oceania', item_type='Household', total_revenue=Decimal('6603844.25'), total_units_sold=9882)
Row(region='Australia and Oceania', item_type='Meat', total_revenue=Decimal('151880.40625'), total_units_sold=360)
Row(region='Europe', item_type='Baby Food', total_revenue=Decimal('2642658.625'), total_units_sold=10352)
Row(region='Europe', item_type='Beverages', total_revenue=Dec

In [51]:
# Step 1: Create the gold table for total sales by country and item type
query_gold_create_2 = """
CREATE TABLE IF NOT EXISTS learning.gold_sales_by_country_item (
    country TEXT,
    item_type TEXT,
    total_revenue DECIMAL,
    total_units_sold INT,
    PRIMARY KEY (country, item_type)
)
"""
session.execute(query_gold_create_2)


In [52]:
from collections import defaultdict

# Step 2: Fetch all relevant data from the silver_sales table
fetch_query_2 = "SELECT country, item_type, total_revenue, units_sold FROM learning.silver_sales"
rows_2 = session.execute(fetch_query_2)

# Step 3: Perform aggregation in Python for the second gold table
aggregated_data_2 = defaultdict(lambda: defaultdict(lambda: {'total_revenue': 0, 'total_units_sold': 0}))

for row in rows_2:
    aggregated_data_2[row.country][row.item_type]['total_revenue'] += row.total_revenue
    aggregated_data_2[row.country][row.item_type]['total_units_sold'] += row.units_sold

# Step 4: Insert aggregated data into the second gold table
for country, items in aggregated_data_2.items():
    for item_type, totals in items.items():
        session.execute("""
        INSERT INTO learning.gold_sales_by_country_item (country, item_type, total_revenue, total_units_sold)
        VALUES (%s, %s, %s, %s)
        """, (country, item_type, totals['total_revenue'], totals['total_units_sold']))

print("Gold table 'gold_sales_by_country_item' populated successfully!")


Gold table 'gold_sales_by_country_item' populated successfully!


In [53]:
query = "SELECT * FROM learning.gold_sales_by_country_item ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(country='Malaysia', item_type='Beverages', total_revenue=Decimal('434357.3125'), total_units_sold=9154)
Row(country='Israel', item_type='Beverages', total_revenue=Decimal('223442.046875'), total_units_sold=4709)
Row(country='Serbia', item_type='Clothes', total_revenue=Decimal('802989.4375'), total_units_sold=7348)
Row(country='Djibouti', item_type='Clothes', total_revenue=Decimal('61415.359375'), total_units_sold=562)
Row(country='Egypt', item_type='Clothes', total_revenue=Decimal('130261.7578125'), total_units_sold=1192)
Row(country='Nicaragua', item_type='Baby Food', total_revenue=Decimal('738014.5'), total_units_sold=2891)
Row(country='Nicaragua', item_type='Household', total_revenue=Decimal('5206491.5'), total_units_sold=7791)
Row(country='Romania', item_type='Beverages', total_revenue=Decimal('217368.453125'), total_units_sold=4581)
Row(country='Romania', item_type='Cereal', total_revenue=Decimal('1509220.875'), total_units_sold=7337)
Row(country='Sao Tome and Principe', item_

In [54]:
# Step 1: Create the gold table for total sales by sales channel and item type
query_gold_create_3 = """
CREATE TABLE IF NOT EXISTS learning.gold_sales_by_sales_channel_item (
    sales_channel TEXT,
    item_type TEXT,
    total_revenue DECIMAL,
    total_units_sold INT,
    PRIMARY KEY (sales_channel, item_type)
)
"""
session.execute(query_gold_create_3)


In [55]:
from collections import defaultdict

# Step 2: Fetch all relevant data from the silver_sales table
fetch_query_3 = "SELECT sales_channel, item_type, total_revenue, units_sold FROM learning.silver_sales"
rows_3 = session.execute(fetch_query_3)

# Step 3: Perform aggregation in Python for the third gold table
aggregated_data_3 = defaultdict(lambda: defaultdict(lambda: {'total_revenue': 0, 'total_units_sold': 0}))

for row in rows_3:
    aggregated_data_3[row.sales_channel][row.item_type]['total_revenue'] += row.total_revenue
    aggregated_data_3[row.sales_channel][row.item_type]['total_units_sold'] += row.units_sold

# Step 4: Insert aggregated data into the third gold table
for sales_channel, items in aggregated_data_3.items():
    for item_type, totals in items.items():
        session.execute("""
        INSERT INTO learning.gold_sales_by_sales_channel_item (sales_channel, item_type, total_revenue, total_units_sold)
        VALUES (%s, %s, %s, %s)
        """, (sales_channel, item_type, totals['total_revenue'], totals['total_units_sold']))

print("Gold table 'gold_sales_by_sales_channel_item' populated successfully!")


Gold table 'gold_sales_by_sales_channel_item' populated successfully!


In [56]:
query = "SELECT * FROM learning.gold_sales_by_sales_channel_item ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(sales_channel='Online', item_type='Baby Food', total_revenue=Decimal('2149968.125'), total_units_sold=8422)
Row(sales_channel='Online', item_type='Beverages', total_revenue=Decimal('1609266.744140625'), total_units_sold=33915)
Row(sales_channel='Online', item_type='Cereal', total_revenue=Decimal('7325799.6875'), total_units_sold=35614)
Row(sales_channel='Online', item_type='Clothes', total_revenue=Decimal('1393210.7578125'), total_units_sold=12749)
Row(sales_channel='Online', item_type='Cosmetics', total_revenue=Decimal('24259791.125'), total_units_sold=55489)
Row(sales_channel='Online', item_type='Fruits', total_revenue=Decimal('331168.3515625'), total_units_sold=35495)
Row(sales_channel='Online', item_type='Household', total_revenue=Decimal('12226667.96875'), total_units_sold=18296)
Row(sales_channel='Online', item_type='Meat', total_revenue=Decimal('15965161.375'), total_units_sold=37842)
Row(sales_channel='Online', item_type='Office Supplies', total_revenue=Decimal('13841468.31